In [1]:
from nltk.tokenize import word_tokenize
from collections import Counter
import collections
import sys
import MeCab
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.model_selection import GridSearchCV

C:\Users\fmv\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#元の文書から名詞限定の文書を用意する
mecab = MeCab.Tagger()
def tokenize(text):
    with open(text, 'r')as f:
        textdata=f.read()#一括で詠み込む  
        textdata = re.split('-{5,}', textdata)[0]+ re.split('-{5,}', textdata)[2]
        textdata = re.split('底本', textdata)[0]
        #詠み込んだ文章を底本部分で分割し本文のみを取得
        textdata = re.split('［＃ここから', textdata)[0]
        #詠み込んだ文章を［＃ここから　部分で分割し本文のみを取得
        lines = textdata.split("\r\n")#改行で分ける
        
        results = []
        for line in lines:
            # 学習に使わない表現の削除処理
            s = line
            s = re.sub('｜', '',s)
            s = re.sub('《.+?》', '', s)# 《》を消す
            s = re.sub('［.+?］', '', s)#[]
            s = re.sub('（.+?）', '', s)#()

            token_list=[]
            node = mecab.parseToNode(s)#parseToNode()を使うと形態素の詳細情報
    
            while node:#ループにして情報を取得していく
                # 語を取得
                word = node.surface#surfaceで表層形
                # 品詞を取得
                part = node.feature.split(",")[0]#featureで形態素情報　[0]で品詞取得

                if part in ["名詞"]:#取得した品詞が一致するなら
                     token_list.append(word)#リストに追加   
                node = node.next
            rl = (" ".join(token_list)).strip()
            results.append(rl)
        w_file = "result_"+text
        with open(w_file, 'w', encoding='utf-8') as wf:
            wf.write("\n".join(results))  

#小泉八雲
tokenize('yukionna.txt') 
tokenize('noctilucae.txt') 
tokenize('hashino_uede.txt') 
tokenize('a_dead_secret.txt') 
tokenize('at_a_railway_station.txt') 
#夏目漱石による小説を5つ程度用意
tokenize('ichiya.txt') 
tokenize('yukionna.txt') 
tokenize('carlyle_hakubutsukan.txt') 
tokenize('gubijinso.txt') 
tokenize('ganjitsu.txt') 
#	太宰 治
tokenize('umi.txt')
tokenize('itto.txt')
tokenize('ichimon_itto.txt')
tokenize('aru_chukoku.txt')
tokenize('anitachi.txt')

In [ ]:
tokenize('umi.txt')
tokenize('itto.txt')
tokenize('ichimon_itto.txt')
tokenize('aru_chukoku.txt')
tokenize('anitachi.txt')

In [5]:
# tf-idf を重みとする単語文書行列に変換
#resultしたものは名詞限定になっている
t1='result_yukionna.txt'
t2='result_noctilucae.txt'
t3='result_hashino_uede.txt'
t4='result_carlyle_hakubutsukan.txt'
t5='result_gubijinso.txt'
t6='result_ganjitsu.txt'
t7='result_umi.txt'
t8='result_itto.txt'
t9='result_ichimon_itto.txt'
t10='result_a_dead_secret.txt'
t11='result_yukionna.txt'
t12='result_aru_chukoku.txt'
t13='result_at_a_railway_station.txt'
t14='result_ichiya.txt'
t15='result_anitachi.txt'


vectorizer = CountVectorizer(input='filename')
corpus=[t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15]  
matrix = vectorizer.fit_transform(corpus)
#CountVectorizer で得られた単語文書行列を，TfidfTransformer でtf-idfの行列に変換する
tfidf = TfidfTransformer()
tfidf_matrix = tfidf.fit_transform(matrix)
np.set_printoptions(precision=2) 

#numpy の配列 ndarray のソーティングより

#vectorizer.get_feature_names() 特徴名（単語）
#tfidf_matrix.toarray()  # tf-idfの行列
name_list = np.array(vectorizer.get_feature_names())
population = np.array(tfidf_matrix.toarray())
print(population)
indices = np.argsort(-population)
#品詞を名詞に限定した場合
#大きい値ほど各文書を特徴付ける単語　 値の多い順に20件表示
print(name_list[indices[::,0:20]])

[[ 0.    0.    0.   ...,  0.    0.    0.  ]
 [ 0.    0.    0.   ...,  0.    0.    0.13]
 [ 0.    0.05  0.   ...,  0.    0.    0.  ]
 ..., 
 [ 0.04  0.    0.   ...,  0.    0.    0.  ]
 [ 0.    0.    0.   ...,  0.09  0.    0.  ]
 [ 0.    0.    0.   ...,  0.    0.    0.  ]]
[['彼女' '巳之吉' '茂作' 'あなた' '小屋' '綺麗' '子供' 'よう' '老人' '渡し守' 'それ' 'お前' '江戸' '吹雪'
  '彼等' 'わし' '約束' '少年' '少女' '自分']
 ['生命' '思惟' '夜光虫' '恒星' '変化' 'よう' '無限' '存在' '天の河' '霊的' '輝き' 'あまた' '水平' '波動'
  '深紅' '小浪' '翠玉' '明滅' '鼓動' '閃光']
 ['わし' 'たち' '平七' '士官' '欄干' '熊本' 'らい' '薩摩' 'よう' '遠く' 'とき' '農家' '鎮台' '薩軍'
  '草鞋' '百姓' '車夫' '灰色' '大砲' '辺り']
 ['カーライル' '婆さん' '倫敦' 'チェルシー' '演説' 'もの' '案内' '細君' 'よう' 'そう' 'これ' '四角' '夫人'
  'チェイン' '往来' '御前' '村夫子' 'わし' 'ほか' 'ここ']
 ['甲野' '反吐' '雅号' 'さん' 'もの' '叡山' '四角' 'どこ' '静か' '馬鹿' '余計' '哲学' '愛嬌' 'ハハハハ'
  'おれ' 'よう' 'だい' '大原女' 'いや' '落第']
 ['元日' '想像' '告白' '新聞' 'もの' '年始' '一昨年' '何処' '具眼' '上手' '正月' '虚子' '屑屋' '世間'
  '去年' '困難' '調子' '読者' 'それ' '紙上']
 ['爆弾' '甲府' 'たち' '三鷹' '子供' 'ひとり' 'そこ' '五能線' '愕然' '東能代' '土地' '津軽' '貴重' '頭上'
  '

### データのスケーリング（正規化）はtf-idfなのでOK

In [7]:
#文書の順番に作者番号が並んだ作者番号リスト（正解ラベルの1次元配列）を作成
doc_info = pd.read_csv('token_files2.csv',encoding='cp932')
#print(doc_info)
files = doc_info['文書'].values
print(files)
author_codes = doc_info['著者コード'].values 
print(author_codes)

['result_yukionna.txt' 'result_noctilucae.txt' 'result_hashino_uede.txt'
 'result_carlyle_hakubutsukan.txt' 'result_gubijinso.txt'
 'result_ganjitsu.txt' 'result_umi.txt' 'result_itto.txt'
 'result_ichimon_itto.txt' 'result_a_dead_secret.txt' 'result_yukionna.txt'
 'result_aru_chukoku.txt' 'result_at_a_railway_station.txt'
 'result_ichiya.txt' 'result_anitachi.txt']
[0 0 0 1 1 1 2 2 2 0 1 2 0 1 2]


In [8]:
#文書ベクトルと作者番号リストを，訓練用とテスト用に分割
#今回は半分に
docs_train    = population[:10]        # 訓練データ
authors_train = author_codes[:10]  # 正解ラベル
docs_test     = population[10:]         # テストデータ
authors_test  = author_codes[10:]   # 正解ラベル

In [20]:
# 分類器を生成
clf =SVC(kernel='linear',  C=1)#パラメータ設定は 
#sklearn.model_selection.GridSearchCV を使った
# 訓練データを使って学習
clf.fit(docs_train, authors_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [21]:
# トレーニングデータを分類
pred_train = clf.predict(docs_train)
print('分類結果:', pred_train)
# 正解率 (accuracy) 
accuracy_train = accuracy_score(authors_train, pred_train)
print('トレーニングデータに対する正解率：{0:.2f}'.format(accuracy_train))

分類結果: [0 0 0 1 1 1 2 2 2 0]
トレーニングデータに対する正解率：1.00


In [22]:
# テストデータに対する精度
pred_test = clf.predict(docs_test)
print('分類結果:', pred_test)
accuracy_test = accuracy_score(authors_test, pred_test)
print('テストデータに対する正解率：{0:.2f}'.format(accuracy_test))

分類結果: [0 0 0 0 0]
テストデータに対する正解率：0.20


In [12]:
conf = confusion_matrix(authors_test, pred_test)
print('混同行列：')
print(conf)

混同行列：
[[2 0]
 [1 1]]


In [13]:
conf = confusion_matrix(authors_train, pred_train)
print('混同行列：')
print(conf)

混同行列：
[[3 0]
 [0 3]]


In [12]:
report = classification_report(y_true=authors_train, y_pred=pred_train)
print('適合率，再現率，F値：')
print(report)

適合率，再現率，F値：
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       1.00      1.00      1.00         3

avg / total       1.00      1.00      1.00         6



In [14]:
report = classification_report(y_true=authors_test, y_pred=pred_test)
print('適合率，再現率，F値：')
print(report)

適合率，再現率，F値：
             precision    recall  f1-score   support

          0       0.67      1.00      0.80         2
          1       1.00      0.50      0.67         2

avg / total       0.83      0.75      0.73         4



In [15]:
Standard = svm.LinearSVC(penalty='l2', loss='hinge', dual=True, tol=1e-3)
LossL2 = svm.LinearSVC(
        penalty='l1', loss='squared_hinge', dual=False, tol=1e-3)
PenaltyL1 = svm.LinearSVC(
        penalty='l2', loss='squared_hinge', dual=True, tol=1e-3)
model_set = [Standard, LossL2, PenaltyL1]

In [16]:
tuned_parameters = [
        {'C': [1, 10, 100, 1000], 'kernel': ['linear']}
]


In [17]:
score = 'f1'
clf = GridSearchCV(
    SVC(), # 識別器
    tuned_parameters, # 最適化したいパラメータセット 
    cv=2, # 交差検定の回数
    scoring='%s_weighted' % score ) # モデルの評価関数の指定

In [18]:
clf.fit(docs_train, authors_train)

C:\Users\fmv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\fmv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\fmv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\fmv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=2, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1_weighted', verbose=0)

In [19]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [ ]:

# 各モデルを一つずつCに関してグリッドサーチを行う
for model in model_set:
    # 5-fold Cross-Validationで評価指標はprecision(適合率)を選択  
    clf = GridSearchCV(model, tuned_parameters, cv=2, scoring='precision')
    # 設定したパラメータで学習しつつ検証を行う
    clf.fit(docs_train, authors_train)

In [ ]:
t1='result_yukionna.txt'
t2='result_noctilucae.txt'
t3='result_hashino_uede.txt'
t4='result_a_dead_secret.txt'
t5='result_at_a_railway_station.txt'

vectorizer = CountVectorizer(input='filename')
corpus=[t1,t2,t3,t4,t5]  
matrix = vectorizer.fit_transform(corpus)
#CountVectorizer で得られた単語文書行列を，TfidfTransformer でtf-idfの行列に変換する
tfidf = TfidfTransformer()
tfidf_matrix = tfidf.fit_transform(matrix)
np.set_printoptions(precision=2) 


#numpy の配列 ndarray のソーティングより

#vectorizer.get_feature_names() 特徴名（単語）
#tfidf_matrix.toarray()  # tf-idfの行列
name_list = np.array(vectorizer.get_feature_names())
population = np.array(tfidf_matrix.toarray())
print(population)
indices = np.argsort(-population)
#品詞を名詞に限定した場合
#大きい値ほど各文書を特徴付ける単語　 値の多い順に20件表示
print(name_list[indices[::,0:20]])

In [ ]:
#model = svm.SVC(kernel='rbf', C=1, gamma=0.1)
#svm.SVC()#SVC(kernel='linear', random_state=None)